In [1]:
import torch
import torch.nn as nn
import torch.optim as optim
import flwr as fl
import numpy as np
import pandas as pd
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from typing import List, Dict, Tuple

# ✅ Set device for computation (GPU if available)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# ✅ Load dataset
file_path = "client_3.csv"  # Adjust path if needed
df = pd.read_csv(file_path)

# ✅ Ensure target column exists
target_column = "TenYearCHD"
if target_column not in df.columns:
    raise ValueError(f"Target column '{target_column}' not found in dataset.")

# ✅ Separate Features and Target
X = df.drop(columns=[target_column]).values
y = df[target_column].values

# ✅ Standardize Features
scaler = StandardScaler()
X = scaler.fit_transform(X)

# ✅ Train/Test Split with stratification
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

# ✅ Convert to PyTorch Tensors
X_train_tensor = torch.tensor(X_train, dtype=torch.float32).to(device)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32).to(device)
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device)
y_test_tensor = torch.tensor(y_test, dtype=torch.long).to(device)

# ✅ Create Custom Dataset & DataLoaders
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y
    
    def __len__(self):
        return len(self.X)
    
    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

def create_dataloaders(batch_size=64):
    train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
    test_dataset = CustomDataset(X_test_tensor, y_test_tensor)
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, test_loader

train_loader, test_loader = create_dataloaders()

# ✅ Define MLP Model
class MLP(nn.Module):
    def __init__(self, input_dim, hidden_dim=128, num_layers=4, dropout=0.2587):
        super(MLP, self).__init__()
        layers = [
            nn.Linear(input_dim, hidden_dim),
            nn.BatchNorm1d(hidden_dim),
            nn.GELU()
        ]
        for _ in range(num_layers - 1):
            layers.extend([
                nn.Linear(hidden_dim, hidden_dim),
                nn.BatchNorm1d(hidden_dim),
                nn.GELU(),
                nn.Dropout(dropout)
            ])
        layers.append(nn.Linear(hidden_dim, 2))
        self.model = nn.Sequential(*layers)
    
    def forward(self, x):
        return self.model(x)

# ✅ Initialize model
input_dim = X_train.shape[1]
model = MLP(input_dim=input_dim).to(device)

# ✅ Define optimizer, loss function, and scheduler
optimizer = optim.Adam(model.parameters(), lr=0.00507)
criterion = nn.CrossEntropyLoss()
scheduler = optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=25)  # T_max matches epochs

# ✅ Fix Parameter Handling
def get_parameters(model: nn.Module) -> List[np.ndarray]:
    return [val.cpu().detach().numpy() for val in model.state_dict().values()]

def set_parameters(model: nn.Module, parameters: List[np.ndarray]) -> None:
    state_dict = model.state_dict()
    new_state_dict = {k: torch.tensor(v).to(device) for k, v in zip(state_dict.keys(), parameters)}
    model.load_state_dict(new_state_dict, strict=True)

# ✅ Define the Federated Learning Client
class FLClient(fl.client.NumPyClient):
    def get_parameters(self, config: Dict[str, str]) -> List[np.ndarray]:
        return get_parameters(model)

    def set_parameters(self, parameters: List[np.ndarray]) -> None:
        set_parameters(model, parameters)

    def fit(self, parameters: List[np.ndarray], config: Dict[str, str]) -> Tuple[List[np.ndarray], int, Dict]:
        self.set_parameters(parameters)
        model.train()
        for epoch in range(25):  # Train for 25 local epochs
            total_loss = 0.0
            for X_batch, y_batch in train_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                optimizer.zero_grad()
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                loss.backward()
                optimizer.step()
                total_loss += loss.item()
            scheduler.step()
            print(f"Epoch {epoch+1}/25, Loss: {total_loss/len(train_loader):.4f}")
        print("Client training complete. Sending updates.")
        return self.get_parameters(config), len(train_loader.dataset), {}

    def evaluate(self, parameters: List[np.ndarray], config: Dict[str, str]) -> Tuple[float, int, Dict]:
        self.set_parameters(parameters)
        model.eval()
        total_loss, correct, total = 0.0, 0, 0
        with torch.no_grad():
            for X_batch, y_batch in test_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                total_loss += loss.item()
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y_batch).sum().item()
                total += y_batch.size(0)
        accuracy = correct / total if total > 0 else 0.0
        print(f"✅ Evaluation: Loss={total_loss/len(test_loader):.4f}, Accuracy={accuracy:.4f}")
        return float(total_loss / len(test_loader)), len(test_loader.dataset), {"accuracy": accuracy}

# ✅ Start the Federated Learning Client
fl.client.start_client(
    server_address="127.0.0.1:8081",
    client=FLClient().to_client(),  # Use .to_client() to avoid deprecation warning
)

	Instead, use the `flower-supernode` CLI command to start a SuperNode as shown below:

		$ flower-supernode --insecure --superlink='<IP>:<PORT>'

	To view all available options, run:

		$ flower-supernode --help

	Using `start_client()` is deprecated.

            This is a deprecated feature. It will be removed
            entirely in future versions of Flower.
        
INFO :      
INFO :      Received: train message d3e83597-ab01-4660-91d7-8deb0567854a


Epoch 1/25, Loss: 0.6487
Epoch 2/25, Loss: 0.5756
Epoch 3/25, Loss: 0.5610
Epoch 4/25, Loss: 0.5471
Epoch 5/25, Loss: 0.5303
Epoch 6/25, Loss: 0.5273
Epoch 7/25, Loss: 0.5082
Epoch 8/25, Loss: 0.5070
Epoch 9/25, Loss: 0.4872
Epoch 10/25, Loss: 0.4701
Epoch 11/25, Loss: 0.4800
Epoch 12/25, Loss: 0.4593
Epoch 13/25, Loss: 0.4428
Epoch 14/25, Loss: 0.4305
Epoch 15/25, Loss: 0.4264
Epoch 16/25, Loss: 0.4069
Epoch 17/25, Loss: 0.3895
Epoch 18/25, Loss: 0.3902
Epoch 19/25, Loss: 0.3695
Epoch 20/25, Loss: 0.3594
Epoch 21/25, Loss: 0.3611
Epoch 22/25, Loss: 0.3566
Epoch 23/25, Loss: 0.3427
Epoch 24/25, Loss: 0.3429


INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 44886422-f64a-4bae-a636-c78526569eca
INFO :      Sent reply
INFO :      
INFO :      Received: train message 268d7c86-0bd6-4c6a-b4b8-8f2338d2a575


Epoch 25/25, Loss: 0.3395
Client training complete. Sending updates.
✅ Evaluation: Loss=0.5445, Accuracy=0.7125
Epoch 1/25, Loss: 0.5143
Epoch 2/25, Loss: 0.5214
Epoch 3/25, Loss: 0.5215
Epoch 4/25, Loss: 0.4958
Epoch 5/25, Loss: 0.4871
Epoch 6/25, Loss: 0.4764
Epoch 7/25, Loss: 0.4711
Epoch 8/25, Loss: 0.4487
Epoch 9/25, Loss: 0.4449
Epoch 10/25, Loss: 0.4395
Epoch 11/25, Loss: 0.4393
Epoch 12/25, Loss: 0.4449
Epoch 13/25, Loss: 0.4366
Epoch 14/25, Loss: 0.4296
Epoch 15/25, Loss: 0.4130
Epoch 16/25, Loss: 0.4236
Epoch 17/25, Loss: 0.4228
Epoch 18/25, Loss: 0.4118
Epoch 19/25, Loss: 0.4184
Epoch 20/25, Loss: 0.3968
Epoch 21/25, Loss: 0.3964
Epoch 22/25, Loss: 0.4073
Epoch 23/25, Loss: 0.3841
Epoch 24/25, Loss: 0.3808


INFO :      Sent reply
INFO :      
INFO :      Received: evaluate message 2a390746-72e5-4767-93dc-ef80bba48a2c


...

INFO :      
INFO :      Received: evaluate message 58308670-e2d0-441a-9de7-b71b7dbd8c9e
INFO :      Sent reply
INFO :      
INFO :      Received: train message 4e627046-eaa2-4d6d-966a-2a73a1fcdbd4


✅ Evaluation: Loss=0.5689, Accuracy=0.7979
Epoch 1/25, Loss: 0.2507
Epoch 2/25, Loss: 0.2249
Epoch 3/25, Loss: 0.1847
Epoch 4/25, Loss: 0.1638
Epoch 5/25, Loss: 0.1839
Epoch 6/25, Loss: 0.1541
Epoch 7/25, Loss: 0.1856
Epoch 8/25, Loss: 0.1531
Epoch 9/25, Loss: 0.1394
Epoch 10/25, Loss: 0.1403
Epoch 11/25, Loss: 0.1260
Epoch 12/25, Loss: 0.1367
Epoch 13/25, Loss: 0.1197
Epoch 14/25, Loss: 0.1016
Epoch 15/25, Loss: 0.0959
Epoch 16/25, Loss: 0.0918
Epoch 17/25, Loss: 0.0634
Epoch 18/25, Loss: 0.0638
Epoch 19/25, Loss: 0.0578
Epoch 20/25, Loss: 0.0675
Epoch 21/25, Loss: 0.0760
Epoch 22/25, Loss: 0.0697
Epoch 23/25, Loss: 0.0709
Epoch 24/25, Loss: 0.0688


INFO :      Sent reply


Epoch 25/25, Loss: 0.0651
Client training complete. Sending updates.


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "Cancelling all calls"
	debug_error_string = "UNKNOWN:Error received from peer ipv4:127.0.0.1:8081 {created_time:"2025-03-17T04:53:27.0617665+00:00", grpc_status:14, grpc_message:"Cancelling all calls"}"
>